In [ ]:
import pandas as pd
from bs4 import BeautifulSoup  
from collections import Counter
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, join
from pandas.io.json import json_normalize
import numpy as np
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [ ]:
####@author:gargla

def cleanup_text(text):
    text = str(text).encode('utf-8').decode('ascii', 'ignore')
    step1Cleaned = BeautifulSoup(text,"lxml")
    import re
    letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
                      " ",                   # The pattern to replace it with
                      step1Cleaned.get_text() )  # The text to search
    lower_case = letters_only.lower()        
    words = lower_case.split()
    words = [w for w in words if not w in stopwords.words("english")]
    return words

##get LIWC features for given word tokens
def get_LIWC_features(words,categories):
    feature_vector={}
    for key,val in categories.items(): 
        itemcounter = {item:count for item, count in [(item, words.count(item)) for item in set(val)]}
        feature_vector[key]=sum(itemcounter.values())
    return feature_vector

def get_data_feature_vector(ResponseDataset):
    columns = ResponseDataset.columns.values
    categoryFileMap=read_categories_fileMap()
    categories = read_LIWC_categories(categoryFileMap)
    #print columns
    count = 0
    feature_vector = {}
    column_ignore_list=["id","batch"]
    ##TODO:add exception handling insted
    user_ignore_list=["usmt12","usmt34","nan","usmt37","ukbr1",
                      "ukbr7","ukbr25","ukbr41","ukbr73","us12","us15",
                      "us39","p3","p9","p18","p35","p41","p46","p53","p62",
                      "p69","p72","p76","p91","j6","j9","j10","j17","j25",
                      "j26","j30","j36","j42","j43","j45","j75","j90","j111",
                     "j113","j115","j117","j118","j120","j124","j126","j127","j128","j130", 'R_1r2C2j6mQ6GiA9k','R_24JjTIBz78Oijl9','R_2aWh2V161nCrCGr','R_cCQ86Yfo950NHzj','R_1PXaNHi9JlURf3b','R_2B8tle2pNhMhrGU','R_3Jltjep2enqKutm','R_1gqIhSkbGH6PRBh','R_3hgUJxvEwwLBr86','R_02LUmCakVkQk9IB','R_1mqvgCSXg8Lxpk2','R_0eKd3rQ9sS4lymt','R_1pX6VSWvUZE8ZhG','R_3kpMaZ2XOUd0xd3','R_231ypXR5P2cIrCq','R_3kuZQv8k2bd9RYC','R_UngyCbt3wGbXPH3','R_AKy9v28nGXPOS9b','R_3m9QTYqCAUFvgKu','R_2abEP94Q6Tbx4kz','R_2zpamsZRsB9r8Q3','R_3nNQ7DXcxNlKJZj','R_2xIRrbvLFp6Rgh0','R_0fGdxXrfIre7itX','R_2VPUUhhERZejlqm','R_3MxuYPQUGjZjzBe','R_2dlMNO2vIJcLpc0','R_1etx1ZroxGgJ0Nv','R_1jVq9aNBh5zCjzQ','R_3kogi5J6i5sYoW6','R_1CdLcMeiGsi4jSt','R_3iK0ImwTmH7s4rL','R_1H658H1dIDY29ds','R_emws4iZFRlrCduV','R_u9tgEqxhepGrplL','R_3PvLHfBO1gZF7as','R_doopIPuO5xvwnVn','R_UAypiJYGI5f5wjf','R_Qn8KsYwgFO6dSHn','R_3MDJomYB4kkhX0H','R_3JfuTdDybtscIvl','R_21H5aQMAsaG7u7a','R_1mqF3wysSzLaG8Z','R_ROcZUZiNGHaKSM9','R_AF36i1gXjpeQaAh','R_2rA5pwmIh4aILzl','R_3g0FGM0DQ9LVdHJ']
    for row in range(ResponseDataset.shape[0]):
    #iterate over each row to get categories for every user response and followups
        userfeatureDict = {}
        user = str(ResponseDataset["id"][row])
        if user in user_ignore_list:
            continue
        print "getting features for id:",user
        for column in columns.tolist():
            if column in column_ignore_list:
                continue
            text = ResponseDataset[column][row]
            words = cleanup_text(text)
            #print words
            features = get_LIWC_features(words,categories)
            features['wordCount']=len(words)
            #print column,":",features
            userfeatureDict[str(column)]=features
        feature_vector[user] = userfeatureDict
    return feature_vector

#pd.DataFrame(featureDict.items(), columns=["text","featurecount"])
    
def create_feature_df(feature_matrix):
    user_ids = []
    frames = []
    for user_id, d in feature_matrix.iteritems():
        user_ids.append(user_id)
        #user_df=pd.DataFrame.from_dict(d, orient='index').stack().reset_index()
        user_df=json_normalize(d)
        frames.append(user_df)
    
    feature_df = pd.concat(frames, keys=user_ids)
    feature_df['id']=list(feature_df.index.levels[0])
    return feature_df

##we have categories as number.csv
###We have a map that translates numbers to the category
def read_categories_fileMap():
    catKeyDict = {}
    with open("LIWCDictKey.csv", 'r') as f:
        for line in f.readlines():
            key,val= line.strip().split(',')
            catKeyDict[int(key)]=val
    return catKeyDict


def read_LIWC_categories(catKeyDict):
    ##TODO:move path to func param
    path="english_dictionary"
    csvs = [f for f in listdir(path) if f.endswith('.csv')]
    categories={}
    for csv in csvs:
        CatList=[]
        with open(path+"/"+csv, 'r') as f:
            for line in f.readlines():
                l= line.strip().split(',')
                CatList.extend(l)
                category,extension=csv.strip().split('.')
        ##get the actual category from number.csv        
        categories[catKeyDict[int(category)]]=CatList
    return categories


def processResponses(responsesFilePath):
    ResponseDf = pd.ExcelFile(responsesFilePath)
    ResponseDataset = ResponseDf.parse(0)
    feature_vector = get_data_feature_vector(ResponseDataset)
    feature_vector_df = create_feature_df(feature_vector)
    return feature_vector_df

In [ ]:
def encodeBinary(x, threshold=4):
    
    """
    encodes value into a binary variable i.e. if value > threshold: return 1, else return 0
    Using this function to encode the entire Personality Dataframe
    
    Arguments: value, threshold(optional)<default=5>
    
    Output: binary value- 0 or 1
    
    """
    if(x>threshold):
        return 1
    else:
        return 0

In [ ]:
def encodeDF(personalityFilePath):
    """
    encodes the personality score data into binary labels.  
    
    Arguments: path for the Personality Score csv file
    
    Output: Data frame with binary labels against each big 5 personality trait for each user
    
    """
    persData=pd.read_csv(personalityFilePath, header=0)
    persData=persData.iloc[:,[0, 131,132,133,134,135]]
    persData=persData[persData.Openness!="#NULL!"] #Removing rows with no Personality form responses
    persData[persData.columns[1:]]=persData[persData.columns[1:]].apply(pd.to_numeric)
    thresholds=persData.quantile(0.7)
    thresholds2=persData.quantile(0.3)
    print(thresholds)
    cols=persData.columns.values[1:]
#     print persData[[cols[0]]].head()
    
    for i,col in enumerate(cols):
        persData.loc[persData[col]<=thresholds2[i],[col]]=0
        persData.loc[persData[col]>=thresholds[i],[col]]=1
#             persData[i]=persData[i].map(encodeBinary)
        
    
    
    return persData

In [ ]:
responsesFilePath="FINALNEWDATA.xlsx"
personalityFilePath="persData.csv"

In [ ]:
liwcData=processResponses(responsesFilePath)
persData=encodeDF(personalityFilePath)

In [ ]:
print liwcData.head(10)
print persData.head(10)

In [ ]:
# Merging LIWC and Personality data to create final dataset

finalData=pd.merge(liwcData,persData, left_on='id', right_on='id', how='left')

In [ ]:
# print finalData.tail(10)
finalData.to_csv("finalDataWithSW.csv")